In [1]:
# import math
# import requests
# from datetime import datetime, timedelta

# import numpy as np
import pandas as pd
import yfinance as yf
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
# from workalendar.usa import UnitedStates
# from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
# from bs4 import BeautifulSoup
# from tqdm import tqdm
# from tqdm.auto import tqdm as tqdm_auto

In [2]:
# start = '2021-01-01' # '2022-01-01'
# end = '2023-12-31'
# symbol = 'AAPL' # "005930.KS"
# fcast_time = 365 # 365일 예측
# freq_option = 'D' 

def get_stock_data(): #start:str='2021-01-01', end:str='2023-12-31', symbol:str='AAPL'
    print(f"{symbol}의 {start}~{end}기간의 주식데이터입니다")
    total_train_df = yf.download(symbol, start, end)
    total_train_df = total_train_df.rename(columns={'Adj Close': 'Adj_Close'})
    
    return total_train_df

def get_stock_data_for_prediction():
    from datetime import datetime, timedelta
    
    # 아직 day를 단위로만 가능하게 설정되어있음
    # 문자열을 datetime 객체로 변환
    if fcast_time <= 0 or not isinstance(fcast_time,int):
        raise ValueError("예측기간은 0보다 큰 정수여야 합니다")
    
    end_date = datetime.strptime(end, '%Y-%m-%d')
    start_date_r = end_date + timedelta(days=1) # 실제 학습에 쓰인 날 마지막 다음날부터하기 위해 end+1
    end_date_r = end_date + timedelta(days=fcast_time+1) # fcast_time+1 만큼의 일수를 더함
    start_r = start_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌
    end_r = end_date_r.strftime('%Y-%m-%d') # 문자로 바꿔줌

    print(f"예측 확인용 데이터는 {symbol}의 {start_r}~{end_r}기간의 주식데이터입니다")
    df_recent = yf.download(symbol, start=start_r, end=end_r)
    df_recent = df_recent.assign(ds = pd.to_datetime(df_recent.index, format = '%Y-%m-%d'))
    df_recent = df_recent.assign(y = df_recent['Close'])
    
    return df_recent
    
def predict_prophet(total_train_df:pd.DataFrame, column_drop:bool=True):
    
    # fcast_time 조건 맞게 설정됐는지
    if fcast_time <= 0 or not isinstance(fcast_time,int):
        raise ValueError("예측기간은 0보다 큰 정수여야 합니다")
    
    # total_train_df['ds'] = pd.to_datetime(total_train_df.index, format = '%Y-%m-%d')
    # total_train_df['y'] = total_train_df['Close']
    total_train_df = total_train_df.assign(ds = pd.to_datetime(total_train_df.index, format = '%Y-%m-%d'))
    total_train_df = total_train_df.assign(y = total_train_df['Close'])
    total_train_df_input = total_train_df[['ds','y']]
    # total_train_df = total_train_df.drop(['ds','y'], axis=1)

    start_year = int(start.split('-')[0])
    range_year = int(end.split('-')[0]) - start_year + 1
    us_holidays = make_holidays_df(
        year_list=[start_year + i for i in range(range_year)], country='US'
    )
    us_holidays = us_holidays.assign(lower_window=-1, upper_window=1)
    
    model_prophet = Prophet(holidays=us_holidays, changepoint_prior_scale=0.3,
                            holidays_prior_scale=10.0, n_changepoints=50, 
                            seasonality_mode='multiplicative', seasonality_prior_scale=0.01).fit(total_train_df_input)

    future = model_prophet.make_future_dataframe(periods = fcast_time+3, freq = freq_option)
    future = future[(future['ds'].dt.dayofweek < 5)&(~future['ds'].isin(us_holidays['ds']))] # 주말 제외 작업 & # 공휴일 제외 작업

    df_forecast = model_prophet.predict(future)

    df_forecast.to_csv('prophet_일년짜리예측(ftn).csv', index=False)
    if column_drop:
        columns_to_drop = ['Christmas Day', 'Christmas Day_lower', 'Christmas Day_upper', 'Christmas Day (observed)', 'Christmas Day (observed)_lower', 
                    'Christmas Day (observed)_upper', 'Columbus Day', 'Columbus Day_lower', 'Columbus Day_upper', 'Independence Day', 
                    'Independence Day_lower', 'Independence Day_upper', 'Independence Day (observed)', 'Independence Day (observed)_lower', 
                    'Independence Day (observed)_upper', 'Juneteenth National Independence Day', 'Juneteenth National Independence Day_lower', 
                    'Juneteenth National Independence Day_upper', 'Juneteenth National Independence Day (observed)', 
                    'Juneteenth National Independence Day (observed)_lower', 'Juneteenth National Independence Day (observed)_upper', 'Labor Day', 
                    'Labor Day_lower', 'Labor Day_upper', 'Martin Luther King Jr. Day', 'Martin Luther King Jr. Day_lower', 
                    'Martin Luther King Jr. Day_upper', 'Memorial Day', 'Memorial Day_lower', 'Memorial Day_upper', "New Year's Day", 
                    "New Year's Day_lower", "New Year's Day_upper", "New Year's Day (observed)", "New Year's Day (observed)_lower", 
                    "New Year's Day (observed)_upper", 'Thanksgiving', 'Thanksgiving_lower', 'Thanksgiving_upper', 'Veterans Day', 'Veterans Day_lower', 
                    'Veterans Day_upper', 'Veterans Day (observed)', 'Veterans Day (observed)_lower', 'Veterans Day (observed)_upper',
                    "Washington's Birthday", "Washington's Birthday_lower", "Washington's Birthday_upper"]

        df_forecast = df_forecast.drop(columns_to_drop, axis=1)
    
    return df_forecast, model_prophet

# table_name = 'predict_aapl_2021-01-01to2023-12-31_to365days'
def Automatically_create_table_names(predict_status:bool=True) -> str:
    if freq_option == 'D': # 후에 딕셔너리 만들어줄까 고민중
        datetime = 'days'
        
    if predict_status == True:
        etc='predict_' #''
        table_name = f'{etc}{symbol}_{start}to{end}_to{str(fcast_time)}{datetime}'
    else:
        table_name = f'{symbol}_{start}to{end}'
    return table_name


def Load_data_in_google_bigquery(df:pd.DataFrame, table_name:str=None, dataset_name:str='stockdata', 
                                 if_exists_option:str='replace', predict_status:bool=False):
    # from google.cloud import bigquery
    # from google.cloud.exceptions import NotFound

    # 초기화(현재 table_name만)
    if table_name is None:
        table_name = Automatically_create_table_names(predict_status)
    
    # BigQuery 클라이언트 생성
    client = bigquery.Client()
    project_id = client.project #'actual-project-415811'

    # 데이터 세트 ID 설정. 이 ID는 '[YOUR_PROJECT_ID].[DATASET_ID]' 형식이어야 합니다.
    dataset_id = "{}.{}".format(project_id, dataset_name)

    # 데이터 세트 참조 생성
    dataset_ref = bigquery.DatasetReference.from_string(dataset_id)

    try:
        # 데이터 세트가 존재하는지 확인
        client.get_dataset(dataset_ref)
        print("Dataset already exists.")
    except NotFound:
        # 데이터 세트가 존재하지 않으면, 데이터 세트 생성
        dataset = bigquery.Dataset(dataset_id) # 데이터 세트 설정
        dataset.location = "asia-northeast3" # 데이터 세트의 지역을 설정(서울로 설정)
        dataset = client.create_dataset(dataset)  # API request # 데이터 세트 생성
        print("Created dataset {}.{}".format(project_id, dataset.dataset_id))

    table_id = "{}.{}".format(dataset_name,table_name)
    
    try:
        df.to_gbq(destination_table=table_id, project_id=project_id, if_exists=if_exists_option)
        print(f"데이터가 성공적으로 BigQuery {dataset_id}에 {table_name}로 로드되었습니다.")
    except Exception as e:
        print(f"데이터 로드 중 에러 발생: {e}")
    
    
def combine_values(df_forecast:pd.DataFrame, df_arima:pd.DataFrame=None):
    start = '2021-01-01'
    end = '2023-12-31'
    symbol = 'AAPL' # "005930.KS"
    
    if df_arima is None:
        # 아리마 데이터
        df_arima = pd.read_csv('arima_forecast.csv')# 'ARIMA_forecast_2024-01.csv')
        df_arima['ds'] = pd.to_datetime(df_arima['ds'])
    
    # 실제값 가져오기
    # get_stock_data() 써도 되지만 적재할 생각은 없는 데이터라 그냥 이것만 해주는게 나았을 것 같아서 안함
    df_merge = yf.download(symbol, start, end)
    # df_merge = df_merge.rename(columns={'Adj Close': 'Adj_Close'})
    df_merge = df_merge.assign(ds = pd.to_datetime(df_merge.index, format = '%Y-%m-%d'))
    # df_merge['y'] = df_merge['Close'] # 마찬가지로 굳이 넣어야하나 고민중
    df_merge = df_merge.reset_index(drop=True)
    df_close = df_merge[['ds','Close']]
    
    # 예측값으로 부터 필요한 부분 가져와서 실제값을 옆에 붙여주기
    df_prophet_concat=df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
    df_prophet_concat2 = df_prophet_concat.merge(df_close, on='ds', how='left')
    # display(df_prophet_concat2.iloc[-252:-248])
    # display(df_prophet_concat2.info())

    # 아리마 데이터에 실제값을 옆에 붙여주기
    df_arima_concat2 = df_arima.merge(df_close, on='ds', how='left') # df_arima_concat 은 없는게 맞음 애초에 컬럼이 그것들 뿐이라 안만듦
    # display(df_arima_concat2.iloc[-22:-18])
    df_arima_concat2 = df_arima_concat2.sort_values(by='ds')

    # 구분자로 model컬럼 추가
    df_prophet_concat2 = df_prophet_concat2.assign(model='prophet')
    df_arima_concat2 = df_arima_concat2.assign(model='arima')

    # 합치기
    combined_data = pd.concat([df_prophet_concat2, df_arima_concat2])
    combined_data = combined_data.reset_index(drop=True)
    
    return combined_data

In [3]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL' # "005930.KS"
fcast_time = 365 # 365일 예측
freq_option = 'D'

total_train_df = get_stock_data()
df_recent = get_stock_data_for_prediction()
df_forecast, model_prophet = predict_prophet(total_train_df)
Load_data_in_google_bigquery(df_forecast) #예측값 

AAPL의 2021-01-01~2023-12-31기간의 주식데이터입니다


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

예측 확인용 데이터는 AAPL의 2024-01-01~2024-12-31기간의 주식데이터입니다



21:34:03 - cmdstanpy - INFO - Chain [1] start processing
21:34:04 - cmdstanpy - INFO - Chain [1] done processing


Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]

데이터가 성공적으로 BigQuery my-personal-project-418208.stockdata에 AAPL_2021-01-01to2023-12-31로 로드되었습니다.


In [4]:
combined_data = combine_values(df_forecast=df_forecast) # combined_data.iloc[950:1000] #.iloc[1000:1020]
Load_data_in_google_bigquery(combined_data, table_name='combined_data')

[*********************100%%**********************]  1 of 1 completed


Dataset already exists.


100%|██████████| 1/1 [00:00<00:00, 999.60it/s]

데이터가 성공적으로 BigQuery my-personal-project-418208.stockdata에 combined_data로 로드되었습니다.


# 모의계산기용

In [5]:
def create_calculator_data():
    start = '2021-01-01'
    end = '2024-12-31'
    symbol = 'AAPL' # "005930.KS"
    df = yf.download(symbol, start, end)
    # df = df.rename(columns={'Adj Close': 'Adj_Close'})
    df['datestamp'] = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df_real = df[['datestamp', 'Close']].rename(columns={'Close': 'y'}).reset_index(drop=True)
    # display(df_real[-1:])
    last_date = df_real['datestamp'].iloc[-1].strftime('%Y-%m-%d')
    # last_date = df_real['datestamp'].max().strftime('%Y-%m-%d') # 정렬이 되어있지 않는 경우
    # last_date = df_real.tail(1)['datestamp'].iloc[0].strftime('%Y-%m-%d')
    
    df_predict = pd.read_csv('prophet_일년짜리예측(ftn).csv')
    df_predict = df_predict[df_predict['ds']>last_date][['ds','yhat']]
    df_predict = df_predict.reset_index(drop=True).rename(columns={'ds': 'datestamp', 'yhat': 'y'})
    # df_predict
    # df_real.info()
    df_predict['datestamp']= pd.to_datetime(df_predict['datestamp'])
    # df_predict.info()
    calculator_data=pd.concat([df_real, df_predict])
    
    return calculator_data

In [6]:
calculator_data = create_calculator_data()
Load_data_in_google_bigquery(calculator_data, table_name='calculator_data')

[*********************100%%**********************]  1 of 1 completed


Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]

데이터가 성공적으로 BigQuery my-personal-project-418208.stockdata에 calculator_data로 로드되었습니다.


# 예측 평가 지표 기록하기

In [7]:
def create_metrics(df_forecast:pd.DataFrame, df_real:pd.DataFrame) -> pd.DataFrame:
    import math
    from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
    
    df_forecast = df_forecast.set_index('ds', drop=False)
    fc = df_forecast[df_forecast['ds'].isin(df_real['ds'])]['yhat']

    mse = mean_squared_error(df_real['y'], fc)
    mae = mean_absolute_error(df_real['y'], fc)
    rmse = math.sqrt(mean_squared_error(df_real['y'], fc))
    mape = mean_absolute_percentage_error(df_real['y'], fc)
    r2score = r2_score(df_real['y'],fc)
    print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, mape: {mape}, r2_score: {r2score}')
    metrics = pd.DataFrame(
        {
        'MSE': [mse],
        'MAE': [mae],
        'RMSE': [rmse],
        'MAPE_percentage': [mape*100],
        'r2_score': [r2score],
        }
    )
    metrics.to_csv('metrics_prophet(ftn).csv', index=False)
    
    return metrics

def combine_metrics(metrics_prophet:pd.DataFrame=None, metrics_arima:pd.DataFrame=None) -> pd.DataFrame:
    if metrics_prophet is None:
        metrics_prophet =pd.read_csv('metrics_prophet(ftn).csv')
    if metrics_arima is None:
        metrics_arima = pd.read_csv('metrics_arima.csv')
    
    metrics_prophet = metrics_prophet.assign(model='prophet')
    metrics_arima = metrics_arima.assign(model='arima')
    metrics_combined = pd.concat([metrics_prophet,metrics_arima])
    
    return metrics_combined

In [8]:
start = '2021-01-01' # '2022-01-01'
end = '2023-12-31'
symbol = 'AAPL'
fcast_time = 30
freq_option = 'D'

total_train_df_metrics = get_stock_data()
df_recent_metrics = get_stock_data_for_prediction()
df_forecast_metrics, model_prophet_metrics = predict_prophet(total_train_df_metrics)
metrics_prophet = create_metrics(df_forecast=df_forecast_metrics, df_real=df_recent_metrics)
Load_data_in_google_bigquery(metrics_prophet, table_name='metrics_prophet_aapl', dataset_name='predict_metrics')
metrics_combined = combine_metrics(metrics_prophet=metrics_prophet)
Load_data_in_google_bigquery(metrics_prophet, table_name='metrics_combined_aapl', dataset_name='predict_metrics')

[*********************100%%**********************]  1 of 1 completed

AAPL의 2021-01-01~2023-12-31기간의 주식데이터입니다
예측 확인용 데이터는 AAPL의 2024-01-01~2024-01-31기간의 주식데이터입니다


[*********************100%%**********************]  1 of 1 completed
21:34:27 - cmdstanpy - INFO - Chain [1] start processing
21:34:28 - cmdstanpy - INFO - Chain [1] done processing


mse: 23.92446578469712, mae: 4.285436554128468, rmse: 4.8912642317397985, mape: 0.02297579536124007, r2_score: -0.22369331269342552
Dataset already exists.


100%|██████████| 1/1 [00:00<00:00, 1001.03it/s]


데이터가 성공적으로 BigQuery my-personal-project-418208.predict_metrics에 metrics_prophet_aapl로 로드되었습니다.
Dataset already exists.


100%|██████████| 1/1 [00:00<?, ?it/s]

데이터가 성공적으로 BigQuery my-personal-project-418208.predict_metrics에 metrics_combined_aapl로 로드되었습니다.
